The following work on data is performed on one specific company - NVDA. However rest of the work will be compatible for any list of companies. The transformations I'll perform on data will be aplicable and remain the same for every stock data avaiable.

In [44]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
import fredapi
from sklearn.impute import KNNImputer
from dotenv import load_dotenv
import os

load_dotenv()

True

First let's start with fetching data from YahooFinance API. The data contains the Close, Open, High, Low and Volume of certain stock until recent day

In [16]:
df_company = yf.download('NVDA','2019-01-01')

[*********************100%***********************]  1 of 1 completed


In [17]:
df_company.head()

Price,Close,High,Low,Open,Volume
Ticker,NVDA,NVDA,NVDA,NVDA,NVDA
Date,,,,,
2019-01-02,3.377779,3.433819,3.224784,3.239414,508752000
2019-01-03,3.173704,3.351495,3.166265,3.317523,705552000
2019-01-04,3.377036,3.415222,3.216106,3.246854,585620000
2019-01-07,3.555819,3.592766,3.382987,3.434316,709160000
2019-01-08,3.467294,3.639630,3.394641,3.637398,786016000


First thing we see is that the data from yahoo api has unusual way of indexing data, so first we are going to fix that

In [ ]:
df_company = df_company.reset_index().rename(columns={"index": "Date"})
df_company_temp = df_company.copy()
df_company_temp.columns = df_company.columns.droplevel(1)
df_company = df_company_temp

Price,Date,Close,High,Low,Open,Volume
0,2019-01-02,3.377779,3.433819,3.224784,3.239414,508752000
1,2019-01-03,3.173704,3.351495,3.166265,3.317523,705552000
2,2019-01-04,3.377036,3.415222,3.216106,3.246854,585620000
3,2019-01-07,3.555819,3.592766,3.382987,3.434316,709160000
4,2019-01-08,3.467294,3.639630,3.394641,3.637398,786016000


In [19]:
df_company.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1558 entries, 0 to 1557
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Date    1558 non-null   datetime64[ns]
 1   Close   1558 non-null   float64       
 2   High    1558 non-null   float64       
 3   Low     1558 non-null   float64       
 4   Open    1558 non-null   float64       
 5   Volume  1558 non-null   int64         
dtypes: datetime64[ns](1), float64(4), int64(1)
memory usage: 73.2 KB


In [20]:
df_company['Date'].head(10)

0   2019-01-02
1   2019-01-03
2   2019-01-04
3   2019-01-07
4   2019-01-08
5   2019-01-09
6   2019-01-10
7   2019-01-11
8   2019-01-14
9   2019-01-15
Name: Date, dtype: datetime64[ns]

From the lines above we can see that there are no NULL's in the yahoo finance datasets and also the values exists only for days when stock market works.

Now let's gather more data that we will use for modelling stock prices. We are going to use macroeconomic indexes from Federak Reserve Economic Data API.

In [45]:
API_KEY = os.getenv('API_KEY')
fred = fredapi.Fred(api_key=API_KEY)


gdp_data = fred.get_series("GDP")
cpi_data = fred.get_series("CPIAUCSL")
unemployment_data = fred.get_series("UNRATE")
fed_funds_data = fred.get_series("FEDFUNDS")
sp500_data = fred.get_series("SP500")
vix_data = fred.get_series("VIXCLS")

df_economic = pd.DataFrame({
    "SP500": sp500_data,
    "VIX": vix_data,
    "GDP": gdp_data,
    "CPI": cpi_data,
    "UR": unemployment_data,
    "FF": fed_funds_data})
    

First thing we notice that is has all the data from the 1946's up to todays date. However there are a lot of NaN's. My guess would be that they fill a certain value at the day they make the calculations.

In [28]:
df_economic.head()

,SP500,VIX,GDP,CPI,UR,FF
1946-01-01,NaN,NaN,NaN,NaN,NaN,NaN
1946-04-01,NaN,NaN,NaN,NaN,NaN,NaN
1946-07-01,NaN,NaN,NaN,NaN,NaN,NaN
1946-10-01,NaN,NaN,NaN,NaN,NaN,NaN
1947-01-01,NaN,NaN,243.164,21.48,NaN,NaN


In [29]:
df_economic.tail()

,SP500,VIX,GDP,CPI,UR,FF
2025-03-07,5770.20,23.37,NaN,NaN,NaN,NaN
2025-03-10,5614.56,27.86,NaN,NaN,NaN,NaN
2025-03-11,5572.07,26.92,NaN,NaN,NaN,NaN
2025-03-12,5599.30,24.23,NaN,NaN,NaN,NaN
2025-03-13,5521.52,NaN,NaN,NaN,NaN,NaN


In [31]:
df_economic.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 9824 entries, 1946-01-01 to 2025-03-13
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   SP500   2515 non-null   float64
 1   VIX     8884 non-null   float64
 2   GDP     312 non-null    float64
 3   CPI     938 non-null    float64
 4   UR      926 non-null    float64
 5   FF      848 non-null    float64
dtypes: float64(6)
memory usage: 537.2 KB


So first thing we do is cleaning the indexes and also filtering the data from 2019-01-01, which will be avaible to select in the function.

In [32]:
df_economic = df_economic.reset_index().rename(columns={"index": "Date"})
df_economic = df_economic[df_economic['Date'] > '2019-01-01']

In [33]:
df_economic.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1637 entries, 8187 to 9823
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Date    1637 non-null   datetime64[ns]
 1   SP500   1558 non-null   float64       
 2   VIX     1579 non-null   float64       
 3   GDP     23 non-null     float64       
 4   CPI     73 non-null     float64       
 5   UR      73 non-null     float64       
 6   FF      73 non-null     float64       
dtypes: datetime64[ns](1), float64(6)
memory usage: 102.3 KB


We notice that there are more dates in df_economic set than on the set obtained from yahoo finances so we merge the data sets with respect for keeping all the dates from df_company. Later using the KNNImputer we impute missing data in macroeconomical indexes.

In [37]:
df_merged = df_merged = pd.merge(df_company, df_economic, on='Date', how='left')
numeric_data = df_merged.drop(columns=['Date']) 
imputer = KNNImputer(n_neighbors=2)
df_merged_imputed = pd.DataFrame(imputer.fit_transform(numeric_data), columns=numeric_data.columns)